# Phase 1: books ids extraction (Nesreen)

In [72]:
CLIENT_ID = 'doDe9gQzQN1nhSLAkBAGQ'
CLIENT_SECRET = 'dZIiAKXZ1gRU1oWxxBy5ED1S6l78B65VgRV2RCS0'

In [73]:
import oauth2

In [74]:
url = 'http://www.goodreads.com'
request_token_url = '%s/oauth/request_token' % url
authorize_url = '%s/oauth/authorize' % url
access_token_url = '%s/oauth/access_token' % url

In [75]:
# consumer authentication using oauth2
consumer = oauth2.Consumer(key= CLIENT_ID, secret= CLIENT_SECRET)
client = oauth2.Client(consumer) #create the client

In [76]:
# The OAuth Client request to get access token
response, content = client.request(request_token_url, 'GET')

In [77]:
if response['status'] != '200':
    raise Exception('Invalid response: %s' % response['status'])

In [78]:
import urllib

In [79]:
dict(urllib.parse.parse_qsl(content))

{b'oauth_token': b'BRx3jDsecno40PTsIVvkpA',
 b'oauth_token_secret': b'Sx1GXwuhohlb39s9N6DSLkuvJvbiUtt1VWORNvf0'}

In [80]:
#get access token    
request_token = dict(urllib.parse.parse_qsl(content))
oauth_token_key,oauth_token_secret = request_token[b'oauth_token'], request_token[b'oauth_token_secret']

In [81]:
from bs4 import BeautifulSoup

In [82]:
from urllib.request import urlopen

In [83]:
books_list = []
#BASE_URL = "https://www.goodreads.com/group/bookshelf/603467-bertelsmann-data-science-book-readers&page=1"
BASE_URL = "https://www.goodreads.com/group/bookshelf/603467-bertelsmann-data-science-book-readers?order=d&per_page=200&shelf=udacians-favorites&sort=date_added&view=main"
html = urlopen(BASE_URL).read()
soup = BeautifulSoup(html,'html.parser')
table = soup.find(id='groupBooks')
for row in table.findAll("tr"):
    if row.td:
        td = row.findAll("td")[0]
        if td.find('a'):
            link = td.find('a')
            books_list.append(link.get('href'))
            continue

# Extract all books ids (Ana)

In [84]:
# imports
import pandas as pd
import re
import goodreads_api_client as gr

In [85]:
# create data frame to hold relevant info
books_df = pd.DataFrame({'link': books_list})

In [86]:
books_df['id'] = books_df['link'].apply(lambda x: re.findall(r'\d+', x)[0])

# Fields extraction 

In [87]:
# connect. Set up connection to Good Reads
gr_client = gr.Client(developer_key='doDe9gQzQN1nhSLAkBAGQ')

In [88]:
book = gr_client.Book.show('1128434')

In [423]:
def try_or_none(obj, attribute):
    try: 
        result = obj[attribute]
        return result
    except: 
        return None

In [424]:
STOP_WORDS = ['to-read', 're-read', 'currently-reading', 'owned', 'books-i-own', 'kindle',
             'favorites']
STOP_STEMS = ['-fantasy'] # not adapted it yet
def take_top_n_meaningful_shelves(shelves: list, n: int) -> list:
    result = [try_or_none(x, '@name') for x in shelves if (try_or_none(x,'@name') not in STOP_WORDS 
                                                           and try_or_none(x,'@name') is not None)][:n]
    return result

In [431]:
NUMBER_OF_FIELDS_FOR_AUTHOR = 9
def extract_fields(book):
    book_id = int(book['id'])
    title = book['title']
    reviews_count = try_or_none(book['work']['reviews_count'],'#text')
    ratings_count = try_or_none(book['work']['ratings_count'],'#text')
    publication_year = try_or_none(book['work']['original_publication_year'],'#text')
    average_rating = float(book['average_rating'])
    ratings_sum = try_or_none(book['work']['ratings_sum'],'#text')
    ratings_count = try_or_none(book['work']['ratings_count'],'#text')
    if (ratings_sum is not None) and (ratings_count is not None) and (int(ratings_count) > 0):
        alt_averate_rating = int(ratings_sum)/int(ratings_count)
    else: 
        alt_averate_rating = None
    n_authors = len(book['authors']['author'])
    single_author = False
    if n_authors == NUMBER_OF_FIELDS_FOR_AUTHOR:
        single_author = True
    if n_authors > 0:
        if single_author:
            author1 = book['authors']['author']
        else:
            author1 = book['authors']['author'][0]
        author1_name = author1['name']
        author1_role = author1['role']
        author1_average_rating = author1['average_rating']
    else: 
        author1 = None
        author1_name = None
        author1_role = None
        author1_average_rating = None
    if (not single_author) and (n_authors > 1):
        author2 = book['authors']['author'][1]
        author2_name = author2['name']
        author2_role = author2['role']
        author2_average_rating = author2['average_rating']
    else: 
        author2 = None
        author2_name = None
        author2_role = None
        author2_average_rating = None
    if (not single_author) and (n_authors > 2):
        author3 = book['authors']['author'][2]
        author3_name = author3['name']
        author3_role = author3['role']
        author3_average_rating = author3['average_rating']
    else:
        author3 = None
        author3_name = None
        author3_role = None
        author3_average_rating = None 
    if (try_or_none(book, 'popular_shelves') is not None) and (try_or_none(book['popular_shelves'], 'shelf') is not None):
        genres = ','.join(take_top_n_meaningful_shelves(book['popular_shelves']['shelf'],10))
    else:
        genres = None                                      
    result = pd.DataFrame({'id': [book_id],
                           'title': [title],
                           'reviews_cnt': [reviews_count],
                           'ratings_cnt': [ratings_count],
                           'pub_year': [publication_year],
                           'avg_rating': [average_rating],
                           'alt_avg_rating': [alt_averate_rating],
                           'author1_name': [author1_name],
                           'author1_role': [author1_role], 
                           'author1_avg_rating': [author1_average_rating],
                           'author2_name': [author2_name],
                           'author2_role': [author2_role], 
                           'author2_avg_rating': [author2_average_rating],
                           'author3_name': [author3_name],
                           'author3_role': [author3_role], 
                           'author3_avg_rating': [author3_average_rating],
                           'genres': [genres] 
    })
    return result

In [441]:
# 38213081, 34874231,7844543, 34109075, 39912728
test_book = gr_client.Book.show(39912728)

In [442]:
extract_fields(test_book)

,id,title,reviews_cnt,ratings_cnt,pub_year,avg_rating,alt_avg_rating,author1_name,author1_role,author1_avg_rating,author2_name,author2_role,author2_avg_rating,author3_name,author3_role,author3_avg_rating,genres
0,39912728,A Tale of Two Cities: Charles Dickens,2,0,None,0.0,None,Charles Dickens,None,3.87,None,None,None,None,None,None,


In [443]:
def build_books_data_frame():
    book = gr_client.Book.show(books_df.id[0])
    result = extract_fields(book)
    for book_id in books_df.id[1:]:
        book = gr_client.Book.show(book_id)
        row = extract_fields(book)
        result = pd.concat([result, row])
    return result

In [444]:
df = build_books_data_frame()

In [446]:
#df

In [447]:
df.to_csv('gr_books_v1.csv', index=False)